In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from  scipy.stats import chi2_contingency
from sklearn.linear_model import LinearRegression

In [2]:
# метод анамарфоз для нормального распределения
def anom_normal_test(df1): 
  g = int(1 + np.floor(np.log2(len(df1))))
  h = (max(df1) - min(df1)) / g
  
  Z = [0] * (g + 1)
  for i in range(len(Z)):
    Z[i] = min(df1) + (h * i)
  data = [0] * g

  for i in range(len(df1)):
    for j in range(len(Z) - 1):
      if df1[i] >= Z[j] and df1[i] <= Z[j + 1]:
        data[j] += 1
  
  mask = [round((Z[i] + Z[i + 1]) / 2, 2) for i in range(len(Z) - 1)]
  range_df = []
  for i in range(len(data)):
    range_df += [mask[i]] * data[i]
  
  mn = 0
  chas = rel_freq(range_df)
  for i in range(len(chas)):
    if chas[i][0] != mask[i]:
      chas = (chas[0 : i] + [[mask[i], 1e-15]] + chas[i:])
      break

  for i in range(len(chas)):
    mn += chas[i][0] * chas[i][1]
  vr = 0
  for i in range(len(chas)):
    vr += ((chas[i][0] - mn) ** 2) * chas[i][1]

  sq = np.sqrt(vr)

  ln_freq = np.log(np.array([i[1] for i in chas]))

  base = [i[0] for i in chas]

  otkl = [0] * g
  for i in range(len(base)):
    otkl[i] = (base[i] - mn) ** 2

  plt.plot(otkl, ln_freq)
  

  otkl = np.array(otkl).reshape((-1, 1))
  ln_freq = np.array(ln_freq)
  
  model = LinearRegression()
  model.fit(otkl, ln_freq)
  a = model.coef_[0]
  b = model.intercept_

  plt.plot([min(otkl), max(otkl)], [min(otkl) * a + b, max(otkl) * a + b])
  mod_ln = (otkl.reshape(-1) * a + b)
  
  
  
  r2 = (np.corrcoef(ln_freq, mod_ln) ** 2)[0][1]
  print('R2 =', r2)